# Parallel computing

### Start and check multiple processes

In [1]:
using BenchmarkTools,  Distributed, LinearAlgebra
D  = Distributed
LA = LinearAlgebra 

LinearAlgebra

In [2]:
D.addprocs(2) # or start julia in a console with the option -p n_processes
# this start multiple processes not threads (processes doesn't share memory as threads does, but threads
# are limited to a single CPU)

2-element Array{Int64,1}:
 2
 3

In [3]:
for pid in D.workers()
    println(pid)
end
    

2
3


In [4]:
D.@everywhere println(D.myid())

1


CompositeException: On worker 2:
UndefVarError: D not defined
top-level scope at none:0
eval at ./boot.jl:319
#116 at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Distributed/src/process_messages.jl:276
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Distributed/src/process_messages.jl:56
run_work_thunk at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Distributed/src/process_messages.jl:65
#102 at ./task.jl:262
#remotecall_wait#154(::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}, ::Function, ::Function, ::Distributed.Worker, ::Module, ::Vararg{Any,N} where N) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Distributed/src/remotecall.jl:407
remotecall_wait(::Function, ::Distributed.Worker, ::Module, ::Vararg{Any,N} where N) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v0.7/Distributed/src/remotecall.jl:398
#remotecall_wait#157(::

### Parallel for -  @parallel for macro
- output_var = @parallel (aggregator) for_cycle 

In [5]:
function f(n)
  s = 0.0
  for i = 1:n
    s += i/2
  end
    return s
end

f (generic function with 1 method)

In [6]:
@time f(100000000)

  0.091672 seconds (16.20 k allocations: 880.640 KiB)


2.500000025e15

In [7]:
function pf(n)
    s = @distributed (+) for i = 1:n # aggregate using sum on variable s   
        i/2                          # last element of for cycle is used by the aggregator
  end
  return s
end

pf (generic function with 1 method)

In [8]:
@time pf(100000000)

  1.184580 seconds (2.39 M allocations: 119.505 MiB, 2.69% gc time)


2.500000025e15

### Parallel map - pmap(f,collection)
 - applies a function f on each elements of the collection

In [9]:
x = [rand(100,100) for i in 1:10];

In [10]:
@benchmark map(LA.svd,x)

BenchmarkTools.Trial: 
  memory estimate:  4.71 MiB
  allocs estimate:  122
  --------------
  minimum time:     35.518 ms (0.00% GC)
  median time:      36.844 ms (0.00% GC)
  mean time:        38.978 ms (1.71% GC)
  maximum time:     93.333 ms (61.08% GC)
  --------------
  samples:          129
  evals/sample:     1

In [11]:
@benchmark D.pmap(LA.svd,x)

BenchmarkTools.Trial: 
  memory estimate:  1.61 MiB
  allocs estimate:  1508
  --------------
  minimum time:     14.535 ms (0.00% GC)
  median time:      14.855 ms (0.00% GC)
  mean time:        15.328 ms (1.61% GC)
  maximum time:     65.235 ms (76.69% GC)
  --------------
  samples:          326
  evals/sample:     1